In [26]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import selenium
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [28]:
def scrape_imdb_reviews(movie_id):
    print('scrapping for id ' + str(movie_id))
    #URL for the movie's reviews page on IMDb
    url = f"https://www.imdb.com/title/{movie_id}/reviews"

    driver = webdriver.Chrome()
    driver.get(url)
    #to wait for the page to load
    time.sleep(2) 

    
    try:
        buttons = driver.find_elements(By.CLASS_NAME, "ipc-btn")
        
        #Initialize a counter for occurrences of "Tout" bc there are 2 buttons "Tout" in general.
        tout_counter = 0
        for button in buttons:
            if button.text.strip() == "Tout":
                tout_counter += 1
            if tout_counter == 2:
                print('activating button Tout')
                driver.execute_script("arguments[0].scrollIntoView();", button)  #Scroll to the button to ensure it's visible
                time.sleep(2)
                button.click()
                break

        #to ensure the reviews load
        time.sleep(10)


        #get the spoilers      
        print("spoilers")
        #page source after interaction
        new_buttons = driver.find_elements(By.CLASS_NAME, "ipc-btn")

        #script to get all the spoilers quickly
        js_script = """
                const buttons = Array.from(document.querySelectorAll('span.ipc-btn__text'));
                buttons.forEach(button => {
                if (button.textContent.trim() === 'Spoiler') {
                    button.click();
                }
            });
        """
        driver.execute_script(js_script)
        
        
        html_from_page = driver.page_source

    
        soup = BeautifulSoup(html_from_page, 'html.parser')

        review_containers = soup.find_all('div', class_='ipc-list-card__content')
        reviews = []
        stars = []
        #0 if not, 1 otherwise
        spoilers = []
        for container in review_containers:
            review_div = container.find('div', class_='ipc-html-content-inner-div')
            review_text = review_div.get_text(strip=True) if review_div else "No review available"
            #review_text = container.find('div', class_='ipc-html-content-inner-div').get_text(strip=True)

            star_rating_tag = container.find('span', class_='ipc-rating-star--rating')
            if star_rating_tag:
                star_rating = star_rating_tag.get_text(strip=True)
            else:
                star_rating = None

            #inch ça marche mdr
            spoiler_tag = container.find('div', class_='ipc-signpost__text')
            spoiler = 0
            if spoiler_tag and spoiler_tag.get_text(strip=True) == 'Spoiler':
                spoiler = 1
        
            reviews.append(review_text)
            stars.append(star_rating)
            spoilers.append(spoiler)

    finally:
        driver.quit()

    return reviews, stars, spoilers

In [9]:
#testing for spoilers
reviews, stars, spoilers = scrape_imdb_reviews('tt0072308')
print(spoilers)

scrapping for id tt0072308
activating button Tout
spoilers
[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]


In [30]:
import pandas as pd
import os

#current working directory
current_dir = os.getcwd()

project_root = os.path.dirname(os.path.dirname(current_dir))
data_dir = os.path.join(project_root, 'data')

csv_file_y = os.path.join(data_dir, 'merged_data.csv')
csv_file_g = os.path.join(data_dir, 'IMDBReviews1.csv')

df_y = pd.read_csv(csv_file_y)
df_g = pd.read_csv(csv_file_g)

print(df_y.head())
print(df_g.head())


         0     tconst               2
0   975900  tt0228333    ghostsofmars
1  9363483  tt0094320   whiteoftheeye
2   261236  tt0083949  awomaninflames
3  6631279  tt0119548      littlecity
4   171005  tt0097499          henryv
   wikipedia_ID            name  release_year  rating  \
0        975900  Ghosts of Mars          2001     6.0   
1        975900  Ghosts of Mars          2001     1.0   
2        975900  Ghosts of Mars          2001     7.0   
3        975900  Ghosts of Mars          2001     7.0   
4        975900  Ghosts of Mars          2001     5.0   

                                       review_detail  
0  Oh no - not more zombies? Oh-yes; there's no e...  
1  Lost my v card to this one. And that's pretty ...  
2  I avoided seeing this film for the longest tim...  
3  Ghosts of Mars: Saw it again last night. I've ...  
4  The special effects for the de-limbing and dec...  


In [31]:
first_column_y = df_y.columns[0]
first_column_g = df_g.columns[0]

#only movies not found by Guillaume
df_filtered = df_y[~df_y[first_column_y].isin(df_g[first_column_g])]

print(df_filtered.head())
print(df_filtered.shape)

           0      tconst                     2
1    9363483   tt0094320         whiteoftheeye
3    6631279   tt0119548            littlecity
6   32456683   tt0061637  diefahnevonkriwojrog
8   20604092  tt13667080            anbuthozhi
10  11448183   tt0367546       bindiyachamkegi
(5524, 3)


In [36]:

expanded_rows = []
c = 1
for index, row in df_filtered.iloc[5200:].iterrows():
    tconst = row['tconst']
    print(c)
    reviews, stars, spoilers = scrape_imdb_reviews(tconst)

    for review, star, spoiler_tag in zip(reviews, stars, spoilers):
        new_row = row.tolist() + [review, star, spoiler_tag]
        expanded_rows.append(new_row)
    c += 1

expanded_df_next = pd.DataFrame(expanded_rows, columns=list(df_filtered.columns) + ['review_summary', 'rating', 'spoiler_tag'])


print(expanded_df_next.head(10))


filename = 'expanded_df_first_100.csv'
expanded_df_next.to_csv(filename, mode='a', index=False, header=not os.path.exists(filename))

1
scrapping for id tt0042436
spoilers
2
scrapping for id tt0318434
spoilers
3
scrapping for id tt0758730
activating button Tout
spoilers
4
scrapping for id tt0001731
spoilers
5
scrapping for id tt0058203
spoilers
6
scrapping for id tt0036714
spoilers
7
scrapping for id tt1424067
spoilers
8
scrapping for id tt0114732
spoilers
9
scrapping for id tt0190984
spoilers
10
scrapping for id tt0198837
spoilers
11
scrapping for id tt0074123
spoilers
12
scrapping for id tt1370372
spoilers
13
scrapping for id tt0249028
spoilers
14
scrapping for id tt0053638
spoilers
15
scrapping for id tt1606214
spoilers
16
scrapping for id tt18364228
spoilers
17
scrapping for id tt0055753
spoilers
18
scrapping for id tt0077131
spoilers
19
scrapping for id tt0756204
spoilers
20
scrapping for id tt0322298
spoilers
21
scrapping for id tt0085991
spoilers
22
scrapping for id tt0025480
spoilers
23
scrapping for id tt0083996
spoilers
24
scrapping for id tt0109351
spoilers
25
scrapping for id tt0057846
spoilers
26
scrappi

In [ ]:
def scrape_reviews(df, filename):
    expanded_rows = []
    for index, row in df.iterrows():
        tconst = row['tconst']
        reviews, stars, spoilers = scrape_imdb_reviews(tconst)
    
        for review, star, spoiler_tag in zip(reviews, stars, spoilers):
            new_row = row.tolist() + [review, star, spoiler_tag]
            expanded_rows.append(new_row)
    
    expanded_df_next = pd.DataFrame(expanded_rows, columns=list(df.columns) + ['review_summary', 'rating', 'spoiler_tag'])
    
    print(expanded_df_next.head())
    expanded_df_next.to_csv(filename, mode='a', index=False, header=not os.path.exists(filename))